In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [10]:
conn.rollback()

userId = '504'
qry = """
select bt.id,	bt."serviceProvider",	bt."type",	bt."amount",	bt."transactionAt",	bt."description",	bt."bankAccountId",	bt."balance",	bt."category",	bt."userDescription",	bt."bsParseJobId",	bt."oPayWalletType", ba.id,	ba."accountNumber",	ba."accountHolderName",	ba."bankId",	ba."userId"
from bank_transaction bt
left join bank_account ba on bt."bankAccountId" = ba.id
where ba."userId" = %s
order by ba.id, bt."transactionAt" ASC, bt.id ASC
        """
cursor.execute(qry, (userId,) )
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./%s-bs.csv' % userId, index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

       id serviceProvider    type  amount             transactionAt  \
0  328385            mono   debit     800 2024-08-06 01:00:00+00:00   
1  328380            mono  credit       2 2024-09-01 01:00:00+00:00   
2  328381            mono   debit  299075 2024-09-01 01:00:00+00:00   
3  328382            mono  credit  300000 2024-09-01 01:00:00+00:00   
4  328383            mono   debit  120075 2024-09-01 01:00:00+00:00   

                                         description  bankAccountId  balance  \
0   SMS/SMS CHARGES JULY 29TH 2024 - AUGUST 2ND 2024            432      392   
1             Interest Paid 01-08-2024 to 31-08-2024            432     1244   
2            MOB/UTO/MARY ABIODUN A/food/25525513811            432     1242   
3  TNF-Paystack/Transfer from CashXpress Nigeria ...            432   300317   
4           MOB/UTO/IYABO OBEDAT A/goods/25520749017            432      317   

  category userDescription bsParseJobId oPayWalletType   id accountNumber  \
0     None     

2025-02-14 18:35:57,553| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
